## Introduction to Spark DataFrame

#### In Apache Spark: 
* **_DataFrame_** is a distributed collection of rows, where each column is named.
* Similar to relational table, Python Pandas object, R dataframe, or Excel sheet with column headers.

#### Similar to RDD:
* Immuatable: DataFrames cannot be changed, only be transformed.
* Lazy evaluation: Task is not executed until an *action* kicks in.
* Distributed:Rows and columns are distributed. 

#### Different from RDD:
* DataFrame is designed to process structured data.
* Query optimization becomes possible. 

### RDD vs DataFrame
<img src="./images/rdd_vs_dataframe.jpg" width="600" height="400" /> 

#### Howe to create a DataFrame:
* Loading data from a file of various formats: JSON, CSV, XML, ...
* Loading data from existing RDD (kind of transformation)
* Loading data from various databases

It can be created using different data formats. For example, loading the data from JSON, CSV.
Loading data from Existing RDD.
Programmatically specifying schema

<img src="./images/DataFrame-in-Spark.png" width="600" height="400" /> 

#### Example: Loading a csv file to DataFrame

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("DataFrame Intro") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

sc = spark.sparkContext

In [3]:
dividends = spark.read.load('data/NYSE_dividends_A.csv', format='csv', header=True, inferSchema=True)
dividends.show(10)
dividends.printSchema()

+--------+------------+-------------------+---------+
|exchange|stock_symbol|               date|dividends|
+--------+------------+-------------------+---------+
|    NYSE|         AIT|2009-11-12 00:00:00|     0.15|
|    NYSE|         AIT|2009-08-12 00:00:00|     0.15|
|    NYSE|         AIT|2009-05-13 00:00:00|     0.15|
|    NYSE|         AIT|2009-02-11 00:00:00|     0.15|
|    NYSE|         AIT|2008-11-12 00:00:00|     0.15|
|    NYSE|         AIT|2008-08-13 00:00:00|     0.15|
|    NYSE|         AIT|2008-05-13 00:00:00|     0.15|
|    NYSE|         AIT|2008-02-13 00:00:00|     0.15|
|    NYSE|         AIT|2007-11-13 00:00:00|     0.15|
|    NYSE|         AIT|2007-08-13 00:00:00|     0.15|
+--------+------------+-------------------+---------+
only showing top 10 rows

root
 |-- exchange: string (nullable = true)
 |-- stock_symbol: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- dividends: double (nullable = true)



In [4]:
dailyPrices= spark.read.load('data/NYSE_daily_prices_A.csv', format='csv', header=True, inferSchema=True)
dailyPrices.show(5)

+--------+------------+-------------------+----------------+----------------+---------------+-----------------+------------+---------------------+
|exchange|stock_symbol|               date|stock_price_open|stock_price_high|stock_price_low|stock_price_close|stock_volume|stock_price_adj_close|
+--------+------------+-------------------+----------------+----------------+---------------+-----------------+------------+---------------------+
|    NYSE|         AEA|2010-02-08 00:00:00|            4.42|            4.42|           4.21|             4.24|      205500|                 4.24|
|    NYSE|         AEA|2010-02-05 00:00:00|            4.42|            4.54|           4.22|             4.41|      194300|                 4.41|
|    NYSE|         AEA|2010-02-04 00:00:00|            4.55|            4.69|           4.39|             4.42|      233800|                 4.42|
|    NYSE|         AEA|2010-02-03 00:00:00|            4.65|            4.69|            4.5|             4.55|      1

### DataFrame Manipulation

You can manipulation a DataFrame in two ways:
1. Using functions of DataFrame
2. Using SQL after creating/registering a table/view 

#### How to Count the number of rows, columns in DataFrame?

In [5]:
dividends.count(), dailyPrices.count()


(8719, 735026)

#### Basic statistics (mean, standard deviance, min ,max , count) of numerical columns

In [7]:
dividends.describe('dividends').show()

+-------+-------------------+
|summary|          dividends|
+-------+-------------------+
|  count|               8719|
|   mean|0.22300571957793303|
| stddev| 0.6983857030438609|
|    min|                0.0|
|    max|             34.958|
+-------+-------------------+



### Basic transformations of DataFrame

1. _select_
2. _filter_
3. _withColumn_
4. _groupBy_
5. _agg_
6. _orderBy_
7. _join_

#### Select column(s) from a DataFrame

In [10]:
dividends.select(['stock_symbol', 'dividends']).show(10)

+------------+---------+
|stock_symbol|dividends|
+------------+---------+
|         AIT|     0.15|
|         AIT|     0.15|
|         AIT|     0.15|
|         AIT|     0.15|
|         AIT|     0.15|
|         AIT|     0.15|
|         AIT|     0.15|
|         AIT|     0.15|
|         AIT|     0.15|
|         AIT|     0.15|
+------------+---------+
only showing top 10 rows



#### Filter the rows 

In [11]:
dailyPrices.filter(dailyPrices.stock_price_close> 200).show()
# equivalentaly
#dailyPrices.filter(dailyPrices['stock_price_close'] > 200).show()
#dailyPrices.filter("stock_price_close > 200").show()

+--------+------------+-------------------+----------------+----------------+---------------+-----------------+------------+---------------------+
|exchange|stock_symbol|               date|stock_price_open|stock_price_high|stock_price_low|stock_price_close|stock_volume|stock_price_adj_close|
+--------+------------+-------------------+----------------+----------------+---------------+-----------------+------------+---------------------+
|    NYSE|         ALX|2010-02-08 00:00:00|          280.05|           280.7|         272.74|           273.24|        4000|               273.24|
|    NYSE|         ALX|2010-02-05 00:00:00|           272.7|          281.32|         272.01|           281.32|       11400|               281.32|
|    NYSE|         ALX|2010-02-04 00:00:00|          278.06|          278.06|         271.55|           271.77|        5300|               271.77|
|    NYSE|         ALX|2010-02-03 00:00:00|          287.73|          287.73|         278.55|           280.06|       

#### Select and Filter

In [12]:
dailyPrices.select('stock_symbol', 'stock_price_close').filter(dailyPrices.stock_price_close> 200).show()

+------------+-----------------+
|stock_symbol|stock_price_close|
+------------+-----------------+
|         ALX|           273.24|
|         ALX|           281.32|
|         ALX|           271.77|
|         ALX|           280.06|
|         ALX|           287.75|
|         ALX|           294.82|
|         ALX|           292.29|
|         ALX|           289.95|
|         ALX|           289.34|
|         ALX|           286.05|
|         ALX|            286.4|
|         ALX|            285.7|
|         ALX|           293.56|
|         ALX|            299.0|
|         ALX|           304.28|
|         ALX|           300.98|
|         ALX|           303.54|
|         ALX|           304.75|
|         ALX|           300.72|
|         ALX|           291.71|
+------------+-----------------+
only showing top 20 rows



#### Mutate by withColumn

In [13]:
augmented = dailyPrices.withColumn('difference', dailyPrices.stock_price_close - dailyPrices.stock_price_open)
augmented.show()

+--------+------------+-------------------+----------------+----------------+---------------+-----------------+------------+---------------------+--------------------+
|exchange|stock_symbol|               date|stock_price_open|stock_price_high|stock_price_low|stock_price_close|stock_volume|stock_price_adj_close|          difference|
+--------+------------+-------------------+----------------+----------------+---------------+-----------------+------------+---------------------+--------------------+
|    NYSE|         AEA|2010-02-08 00:00:00|            4.42|            4.42|           4.21|             4.24|      205500|                 4.24|-0.17999999999999972|
|    NYSE|         AEA|2010-02-05 00:00:00|            4.42|            4.54|           4.22|             4.41|      194300|                 4.41|-0.00999999999999...|
|    NYSE|         AEA|2010-02-04 00:00:00|            4.55|            4.69|           4.39|             4.42|      233800|                 4.42| -0.1299999999

#### GroupBy, Aggregate, and OrderBy

In [16]:
dailyPrices.groupBy('stock_symbol').agg({'stock_price_close': 'max'}).orderBy('max(stock_price_close)', ascending=False).show(10)

+------------+----------------------+
|stock_symbol|max(stock_price_close)|
+------------+----------------------+
|         ALX|                467.25|
|         ADI|                182.62|
|         ACL|                175.47|
|         AXP|                 169.0|
|         AZO|                166.82|
|         AIG|                 156.5|
|         AET|                153.93|
|         AVB|                148.52|
|         AEG|                147.58|
|         APA|                 146.8|
+------------+----------------------+
only showing top 10 rows



In [19]:
joined_df = dailyPrices.join(dividends, ['stock_symbol', 'date'], 'inner').select('stock_symbol','date', 'stock_price_close', 'dividends')

In [20]:
joined_df.show(10)

+------------+-------------------+-----------------+---------+
|stock_symbol|               date|stock_price_close|dividends|
+------------+-------------------+-----------------+---------+
|         AEA|2009-11-20 00:00:00|             6.24|    0.063|
|         AEA|2009-08-21 00:00:00|              5.9|    0.063|
|         AEA|2009-05-21 00:00:00|             4.35|    0.063|
|         AEA|2009-02-20 00:00:00|             1.01|    0.063|
|         AEA|2008-11-21 00:00:00|              1.5|    0.063|
|         AEA|2008-08-22 00:00:00|             4.94|    0.125|
|         AEA|2008-05-22 00:00:00|             6.99|    0.125|
|         AEA|2008-02-22 00:00:00|             7.07|    0.125|
|         AEA|2007-11-23 00:00:00|             8.37|    0.125|
|         AEA|2007-08-24 00:00:00|            12.86|    0.125|
+------------+-------------------+-----------------+---------+
only showing top 10 rows



In [ ]:
#df = df1.join(df2.withColumnRenamed('y1','x1').withColumnRenamed('y2','x2'), ['x1','x2'])

### Running SQL Queries

The *sql* function enables applications to run SQL queries and returns the result as a DataFrame.

* Global Temporary View


In [21]:
dailyPrices.createOrReplaceTempView('daily_prices')
dividends.createOrReplaceTempView('dividends')

In [23]:
price_result = spark.sql('SELECT * FROM daily_prices LIMIT 10')
price_result.show(10)



#result.filter(result['stock_price_close'] > 2).show()
#result_1 = result.rdd.map(lambda row: (row,1)).toDF()
#result_1.show()

+--------+------------+-------------------+----------------+----------------+---------------+-----------------+------------+---------------------+
|exchange|stock_symbol|               date|stock_price_open|stock_price_high|stock_price_low|stock_price_close|stock_volume|stock_price_adj_close|
+--------+------------+-------------------+----------------+----------------+---------------+-----------------+------------+---------------------+
|    NYSE|         AEA|2010-02-08 00:00:00|            4.42|            4.42|           4.21|             4.24|      205500|                 4.24|
|    NYSE|         AEA|2010-02-05 00:00:00|            4.42|            4.54|           4.22|             4.41|      194300|                 4.41|
|    NYSE|         AEA|2010-02-04 00:00:00|            4.55|            4.69|           4.39|             4.42|      233800|                 4.42|
|    NYSE|         AEA|2010-02-03 00:00:00|            4.65|            4.69|            4.5|             4.55|      1

In [ ]:
dividend_result = spark.sql('SELECT * FROM dividends')
dividend_result.show()


#### Join on two views
* List the closing prices when companies paid dividends

In [ ]:
join = spark.sql('''SELECT div.exchange, div.stock_symbol, div.date, div.dividends,
prices.stock_price_close  FROM dividends div INNER JOIN daily_prices prices
ON(div.stock_symbol=prices.stock_symbol AND div.date=prices.date) LIMIT 10''')
join.show()


##### Join and GroupBy 
* What are the maximum, minimum, and average closing procies at the time of dividends

In [ ]:
join_group = spark.sql('''SELECT div.stock_symbol, max(prices.stock_price_close) as max_close FROM dividends div 
INNER JOIN daily_prices prices ON(div.stock_symbol=prices.stock_symbol AND div.date=prices.date)
GROUP BY div.stock_symbol LIMIT 10''')
join_group.show()
    

In [ ]:
join_group_agg = spark.sql('''SELECT div.stock_symbol, max(prices.stock_price_close) maximum,
min(prices.stock_price_close) minimum, avg(prices.stock_price_close) average FROM dividends div 
INNER JOIN daily_prices prices ON(div.stock_symbol=prices.stock_symbol AND div.date=prices.date) 
GROUP BY div.stock_symbol LIMIT 10''')
join_group_agg.show()

In [ ]:
result = join_group_agg.collect()

In [ ]:
for item in result:
    print(item)